In [ ]:
'''Application Building:
    After the model is built, integrating it to a web application
    using Flask framework so that normal users can also use it 
    to predict the energy in a no-code manner. In the application,
    the user provides the required values and get the prediction.
    user provides input as city and get the weather conditions for that city
    such as temperature, wind direction, wind speed from openweather API and user can also
    give the particular month,day to predict the output energy power for a given month and day'''

In [ ]:
import numpy as np
from flask import Flask, request, jsonify, render_template
import joblib
import requests

app = Flask(__name__)
model = joblib.load('xg_RFR_forecast_model.sav')
@app.route('/')
def home():
    return render_template('intro.html')
@app.route('/predict')
def predict():
    return render_template('predict_page.html')
@app.route('/windapi',methods=['POST'])
def windapi():
    city=request.form.get('city')
    apikey="e26bb531d3393dec23475ee08ea9559b"
    url="http://api.openweathermap.org/data/2.5/weather?q="+city+"&appid="+apikey
    resp = requests.get(url)
    resp=resp.json()
    temp = str((resp["main"]["temp"])-273.15) +" °C"
    humid = str(resp["main"]["humidity"])+" %"
    pressure = str(resp["main"]["pressure"])+" mmHG"
    speed = str((resp["wind"]["speed"])*3.6)+" Km/s"
    direc = str((resp["wind"]["deg"]))+" deg"
    return render_template('predict_page.html', temp=temp, humid=humid, pressure=pressure,speed=speed,direction = direc)   
@app.route('/y_predict',methods=['POST'])
def y_predict():
    x_test =[]
    month_dic = {"jan":1,"feb":2,"mar":3,"apr":4,"may":5,"jun":6,"jul":7,"aug":8,"sept":9,"oct":10,"nov":11,"dec":12}
    m = request.form['month']
    print(m)
    x_test.append(month_dic[m])
    d = int(request.form['day'])
    x_test.append(d)
    t = float(request.form['temp'])
    x_test.append(t)
    direc = float(request.form['direc'])
    x_test.append(direc)
    speed= float(request.form['wind'])
    x_test.append(speed)
    x_test = [x_test]
    print(x_test)
    prediction = model.predict(x_test)
    print(prediction)
    output = prediction[0] 
    return render_template('predict_page.html', prediction_text='The energy predicted is {:.2f} KWh'.format(output))


if __name__ == "__main__":
    app.run(debug=False)